In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3..3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.23.jar

--2021-07-17 16:03:59--  https://jdbc.postgresql.org/download/postgresql-42.2.23.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1005522 (982K) [application/java-archive]
Saving to: ‘postgresql-42.2.23.jar.3’

postgresql-42.2.23. 100%[===================>] 981.96K  5.72MB/s    in 0.2s    

2021-07-17 16:04:00 (5.72 MB/s) - ‘postgresql-42.2.23.jar.3’ saved [1005522/1005522]



In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SpotifyAnalysis").config("spark.jars", "postgresql-42.2.23.jar.2") \
    .getOrCreate()

In [4]:
from pyspark import SparkFiles

url = 'https://spotify-bucket-washu.s3.us-east-2.amazonaws.com/tracks.csv'
spark.sparkContext.addFile(url)


spotify_df = spark.read.option('header', 'true').csv(SparkFiles.get("tracks.csv"), inferSchema=True, timestampFormat="mm/dd/yy")
spotify_df.show(10)

+--------------------+--------------------+----------+-----------+--------+-------------------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+
|                  id|                name|popularity|duration_ms|explicit|            artists|          id_artists|release_date|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|
+--------------------+--------------------+----------+-----------+--------+-------------------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+
|35iwgR4jXetI318WE...|               Carve|         6|     126903|       0|            ['Uli']|['45tIt06XoI0Iio4...|  1922-02-22|       0.645| 0.445|  0| -13.338|   1|      0.451|       0.674|           0.744|   0.151|  0.127|104.851|             3|


In [5]:
pop_df = spotify_df.select(['name','artists','popularity','explicit', 'danceability','loudness', 'speechiness','acousticness','instrumentalness', 'tempo'])
pop_df.show(20)

+--------------------+-------------------+----------+--------+------------+--------+-----------+------------+----------------+-------+
|                name|            artists|popularity|explicit|danceability|loudness|speechiness|acousticness|instrumentalness|  tempo|
+--------------------+-------------------+----------+--------+------------+--------+-----------+------------+----------------+-------+
|               Carve|            ['Uli']|         6|       0|       0.645| -13.338|      0.451|       0.674|           0.744|104.851|
|Capítulo 2.16 - B...|['Fernando Pessoa']|         0|       0|       0.695| -22.136|      0.957|       0.797|             0.0|102.009|
|Vivo para Querert...|['Ignacio Corsini']|         0|       0|       0.434|  -21.18|     0.0512|       0.994|          0.0218|130.418|
|El Prisionero - R...|['Ignacio Corsini']|         0|       0|       0.321| -27.961|     0.0504|       0.995|           0.918| 169.98|
| Lady of the Evening|    ['Dick Haymes']|         0|  

In [6]:
total_df = pop_df.groupby('name').agg({'popularity':"count", 'acousticness':"avg",'danceability':'avg'})
total_df.show()

+--------------------+-----------------+------------------+------------------+
|                name|count(popularity)| avg(acousticness)| avg(danceability)|
+--------------------+-----------------+------------------+------------------+
|Need a Little Sug...|                1|             0.992|             0.693|
|Chapter 2.6 - Kró...|                1|             0.735|              0.73|
|Chapter 3 - The U...|                1|             0.629|             0.626|
|Chapter 1.1 - O J...|                1|             0.567|             0.672|
|           Bob White|                3|0.8703333333333334|             0.663|
|I've Got a Feelin...|                9|0.8351111111111112|0.5017777777777778|
|Everybody Loves M...|                1|             0.996|             0.618|
|Chapter 51 - The ...|                1|             0.645|             0.678|
|How Deep Is the O...|                2|             0.982|            0.4615|
|      Sidewalk Blues|                2|            

In [7]:
acous_df = total_df.groupby("name").agg({"acousticness":"count"})\

acous_df.orderBy("count(acousticness)").show(truncate=False)

AnalysisException: ignored

In [8]:
from pyspark.sql.types import * 

In [9]:
final_df = pop_df.withColumn("popularity",pop_df["popularity"].cast(IntegerType()))\
    .withColumn("explicit",pop_df["explicit"].cast(IntegerType()))\
    .withColumn("danceability",pop_df["danceability"].cast(IntegerType()))\
    .withColumn("loudness",pop_df["loudness"].cast(IntegerType()))\
    .withColumn("speechiness",pop_df["speechiness"].cast(IntegerType()))\
    .withColumn("acousticness",pop_df["acousticness"].cast(IntegerType()))\
    .withColumn("instrumentalness",pop_df["instrumentalness"].cast(IntegerType()))\
    .withColumn("tempo",pop_df["tempo"].cast(IntegerType()))

final_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- explicit: integer (nullable = true)
 |-- danceability: integer (nullable = true)
 |-- loudness: integer (nullable = true)
 |-- speechiness: integer (nullable = true)
 |-- acousticness: integer (nullable = true)
 |-- instrumentalness: integer (nullable = true)
 |-- tempo: integer (nullable = true)



In [ ]:
#  Configure settings for RDS
# mode = "append"
# jdbc_url="jdbc:postgresql://final-project-team2.chbl5fhaiumv.ca-central-1.rds.amazonaws.com:5432/spotify_db"
# config = {"user":"postgres", 
#           "password": "kerrynourbrian", 
#          "driver":"org.postgresql.Driver"}

In [19]:
final_df.write.format("jdbc") \
.option("url", "jdbc:postgresql://final-project-team2.chbl5fhaiumv.ca-central-1.rds.amazonaws.com:5432/spotify_db") \
.option("dbtable", "spotify") \
.option("user", "postgres") \
.option("password", "kerrynourbrian") \
.option("driver", "org.postgresql.Driver").mode('append').save()

In [18]:
final_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- explicit: integer (nullable = true)
 |-- danceability: integer (nullable = true)
 |-- loudness: integer (nullable = true)
 |-- speechiness: integer (nullable = true)
 |-- acousticness: integer (nullable = true)
 |-- instrumentalness: integer (nullable = true)
 |-- tempo: integer (nullable = true)



In [22]:
# Write DataFrame to active_user table in RDS

final_df.write.jdbc(url=url, table='spotify', properties=config)

NameError: ignored